In [1]:
import PyPDF2

In [2]:
pdf = PyPDF2.PdfFileReader('data/src/pdf/sample1.pdf')

In [3]:
print(pdf.isEncrypted)

False


In [4]:
src_pdf = PyPDF2.PdfFileReader('data/src/pdf/sample1.pdf')

In [5]:
dst_pdf = PyPDF2.PdfFileWriter()

In [6]:
dst_pdf.cloneReaderDocumentRoot(src_pdf)

In [7]:
print(src_pdf.documentInfo)

{'/Title': IndirectObject(33, 0), '/Producer': IndirectObject(34, 0), '/Creator': IndirectObject(35, 0), '/CreationDate': IndirectObject(36, 0), '/ModDate': IndirectObject(36, 0)}


In [8]:
# dst_pdf.addMetadata(src_pdf.documentInfo)
# TypeError: createStringObject should have str or unicode arg

In [9]:
d = {key: src_pdf.documentInfo[key] for key in src_pdf.documentInfo.keys()}

In [10]:
print(d)

{'/Title': 'sample1', '/Producer': 'macOS バージョン10.14.2（ビルド18C54） Quartz PDFContext', '/Creator': 'Keynote', '/CreationDate': "D:20190114072947Z00'00'", '/ModDate': "D:20190114072947Z00'00'"}


In [11]:
dst_pdf.addMetadata(d)

In [12]:
dst_pdf.encrypt('password')

In [13]:
with open('data/temp/sample1_pass.pdf', 'wb') as f:
    dst_pdf.write(f)

In [14]:
def set_password(src_path, dst_path, password):
    src_pdf = PyPDF2.PdfFileReader(src_path)
    
    dst_pdf = PyPDF2.PdfFileWriter()
    dst_pdf.cloneReaderDocumentRoot(src_pdf)
    
    d = {key: src_pdf.documentInfo[key] for key in src_pdf.documentInfo.keys()}
    dst_pdf.addMetadata(d)
    
    dst_pdf.encrypt(password)
    
    with open(dst_path, 'wb') as f:
        dst_pdf.write(f)

In [15]:
set_password('data/src/pdf/sample1.pdf', 'data/temp/sample1_pass.pdf', 'password')

In [16]:
pdf_rc4 = PyPDF2.PdfFileReader('data/src/pdf/sample1_pass_rc4.pdf')

In [17]:
print(pdf_rc4.isEncrypted)

True


In [18]:
# print(pdf_rc4.documentInfo)
# PdfReadError: file has not been decrypted

In [19]:
print(pdf_rc4.decrypt('wrong-password'))

0


In [20]:
print(pdf_rc4.decrypt('password'))

1


In [21]:
print(pdf_rc4.documentInfo)

{'/Producer': 'macOS バージョン10.14.2（ビルド18C54） Quartz PDFContext', '/Title': 'sample1', '/Creator': 'Keynote', '/CreationDate': "D:20190114072947Z00'00'", '/ModDate': "D:20190114072947Z00'00'"}


In [22]:
pdf_aes = PyPDF2.PdfFileReader('data/src/pdf/sample1_pass_aes.pdf')

In [23]:
# print(pdf_aes.decrypt('password'))
# NotImplementedError: only algorithm code 1 and 2 are supported

In [24]:
def change_password(src_path, dst_path, src_password, dst_password=None):
    src_pdf = PyPDF2.PdfFileReader(src_path)
    src_pdf.decrypt(src_password)
    
    dst_pdf = PyPDF2.PdfFileWriter()
    dst_pdf.cloneReaderDocumentRoot(src_pdf)
    
    d = {key: src_pdf.documentInfo[key] for key in src_pdf.documentInfo.keys()}
    dst_pdf.addMetadata(d)
    
    if dst_password:
        dst_pdf.encrypt(dst_password)
    
    with open(dst_path, 'wb') as f:
        dst_pdf.write(f)

In [25]:
change_password('data/src/pdf/sample1_pass_rc4.pdf', 'data/temp/sample1_no_pass.pdf', 'password')

In [26]:
change_password('data/src/pdf/sample1_pass_rc4.pdf', 'data/temp/sample1_new_pass.pdf',
                'password', 'new_password')